In [1]:
# Solving the "gravitational settling in two dimensions" problem in Lautrup p. 182
import numpy as np
import matplotlib.pyplot as plt
from fenics import * 

# Physical constants 
g0     = 9.8
rho    = 1/9.8 # such that g0*rho = 1
E, nu  = 2, 1.0/3

# ... convert to Lamé paramenters (Lautrup p. 129)
mu  = E/(2+2*nu)
lam = E*nu/((1+nu)*(1-2*nu))

ModuleNotFoundError: No module named 'fenics'

In [ ]:
# The 2D mesh
W, H = 1, 1  # Length, Height (m)
Wres, Hres = 10, 10 # Resolution
mesh = RectangleMesh(Point(0,0), Point(W,H), Wres, Hres)

plt.figure()
plot(mesh, lw=0.75); 

In [ ]:
# Function space
deg = 1 # Polynomial degree 
V   = VectorFunctionSpace(mesh, 'Lagrange', deg); # Polynomial function space of order "deg" (=1 is linear, ...)
u   = TrialFunction(V) # the unknown function
w   = TestFunction(V)  # the weight function 

# The FEM problem
fvec = rho*Constant((0,-g0)) # Gravity force vector
def eps(u): return sym(grad(u)) # strain tensor (symmetric gradient of displacement field, i.e. 2x2 matrix)
def sig(u): return 2*mu*eps(u) + lam*Identity(2)*tr(eps(u))
a = inner(sig(u), grad(w)) * dx # "inner" is the double dot product
L = dot(fvec,w) * dx            # "dot" is the regular (single) dot product 
# Note no boundary terms! w_i=0 where u_i is specified, and we assume no shear stresses which removes the remaining contributions (see slides).

# Define the 4 boundaries
# Note "x = x[0]" is first spatial dimension, and "y = x[1]" is the second spatial dimension
def bot_boundary(x, on_boundary): return on_boundary and near(x[1], 0) 
def top_boundary(x, on_boundary): return on_boundary and near(x[1], H) 
def lft_boundary(x, on_boundary): return on_boundary and near(x[0], 0) 
def rht_boundary(x, on_boundary): return on_boundary and near(x[0], W) 

# Boundary conditions 
bc  = [] # empty list of boundary conditions that we can append (add) to
#bc += [DirichletBC(V.sub(0), Constant(0), lft_boundary)] # u_x = 0 on lft_boundary
#bc += [DirichletBC(V.sub(1), Constant(0), bot_boundary)] # u_y = 0 on bot_boundary
# ... or if no displacement at all on left and bottom boundaries (uncomment below, comment above)
bc += [DirichletBC(V, Constant((0,0)), lft_boundary)] # u = 0 on lft_boundary
bc += [DirichletBC(V, Constant((0,0)), bot_boundary)] # u = 0 on bot_boundary

# Compute solution
usol = Function(V)       # Define a new function in the space "V" with expansion coefs = 0.
solve(a == L, usol, bc); # Solve the linear FEM problem (Ax=b) for the unknown expansion coef. vector (x) and stored it in "usol".


In [ ]:
plt.figure()
h = plot(usol, mode='displacement', title='Magnitude of displacement vector (color)'); # Plot solution
plot(mesh); 
plt.colorbar(h);

# Calculate pressure field
plt.figure()
p = -tr(sig(usol))/2 # mean normal stress
h = plot(p, title='Pressure');
plt.colorbar(h);

In [ ]:
# Potential energy contributions (elastic and gravitational) (see p. 136)
U_ela = assemble( 0.5*inner(sig(usol),eps(usol)) * dx ) # Integral over the elastic energy density
dU_pot = assemble( -dot(fvec,usol) * dx )               # Integral over the change in grav. potential energy density
print('U_ela =', U_ela)
print('dU_pot =', dU_pot)